# Лабораторная 2
## Подраздел: Решение системы нелинейных уравнений

* Cтудент: Ефимов А.В.
* Группа: М8О-307Б
* Вариант: 7

## Задание

Реализовать методы простой итерации и Ньютона решения систем нелинейных уравнений
в виде программного кода, задавая в качестве входных данных точность вычислений.
С использованием разработанного программного обеспечения решить систему нелинейных уравнений
(при наличии нескольких решений найти то из них, в котором значения неизвестных являются 
положительными); начальное приближение определить графически.
Проанализировать зависимость погрешности вычислений от количества итераций.  

## Решение

Загрузка библиотек, настройка лог сообщений и задание функций:

In [1]:
import logging, argparse
import numpy as np
from math import sqrt, exp, log
from numpy.linalg import solve

# Configurations
np.set_printoptions(suppress=True)
logging.basicConfig(level=logging.DEBUG)

MAX_ITER = 1000

f = [
    lambda x : x[0] ** 2 + x[1] ** 2 - 4,
    lambda x : x[0] - exp(x[1]) + 2
]

df = [
    {
        "x0": lambda x : 2 * x[0],
        "x1": lambda x : 2 * x[1]
    },
    {
        "x0": lambda x : 1,
        "x1": lambda x : -exp(x[1])
    }
]

phi = [
    lambda x : sqrt(4 - x[1]**2),
    lambda x : log(x[0] + 2)
]

dphi = [
    {
        "x0": lambda x : 0,
        "x1": lambda x : - x[1] / sqrt(4 - x[1] ** 2)
    },
    {
        "x0": lambda x : 1 / (x[0] + 2),
        "x1": lambda x : 0
    }
]

Вычисление нормали разницы двух векторов (в частности, векторов новых и старых значений):

In [2]:
def norm(x, x_prev):
    return sqrt(sum([(xn - xp) ** 2 for xn, xp in zip(x, x_prev)]))

### Итеративный метод

In [3]:
def iteration_method(a, b, eps):
    def get_phi_norm(x):
        return max(abs(dphi[0]["x0"](x)) + abs(dphi[0]["x1"](x)),
                   abs(dphi[1]["x0"](x)) + abs(dphi[1]["x1"](x)))

    x0_interv = [a[0], b[0]]
    x1_interv = [a[1], b[1]]

    x_prev = [
            (x0_interv[1] + x0_interv[0]) / 2,
            (x1_interv[1] + x1_interv[0]) / 2
    ]

    q = get_phi_norm(x_prev)

    if (q >= 1):
        logging.warning(f"q >= 1 (equals {q}), cannot estimate root")
        return None

    logging.info(f"Inital x0 = {x_prev[0]}, x1 = {x_prev[1]}, q = {q}")
    
    iter_no = 0
    while iter_no <= MAX_ITER:
        iter_no += 1

        x = [func(x_prev) for func in phi]

        logging.info(f"Iteration {iter_no}: x0 = {x[0]}, x1 = {x[1]}")

        error = q / (1 - q) * norm(x, x_prev)
        if (error <= eps):
            break

        logging.info(f"{error} > {eps} , continue...")
        x_prev = x

    logging.info(f"Method ended on iteration {iter_no} with x0 value of {x[0]}, x1 value of {x[1]}")
    return x

### Метод Ньютона

In [4]:
def newton_method(a, b, eps):

    def jacobi_matrix(x):
        return [
            [df[0]["x0"](x), df[0]["x1"](x)],
            [df[1]["x0"](x), df[1]["x1"](x)]
        ]

    x0_interv = [a[0], b[0]]
    x1_interv = [a[1], b[1]]

    x_prev = [
            (x0_interv[1] + x0_interv[0]) / 2,
            (x1_interv[1] + x1_interv[0]) / 2
    ]

    logging.info(f"Inital x0 = {x_prev[0]}, x1 = {x_prev[1]}")

    iter_no = 0
    while iter_no <= MAX_ITER:
        iter_no += 1

        jacobi = np.array(jacobi_matrix(x_prev))
        b = np.array([-f[0](x_prev), -f[1](x_prev)])
        delta_x = solve(jacobi, b).tolist()

        x = [px + dx for px, dx in zip(x_prev, delta_x)]

        logging.info(f"Iteration {iter_no}: x0 = {x[0]}, x1 = {x[1]}")

        error = norm(x, x_prev)
        if (error <= eps):
            break

        logging.info(f"{error} > {eps} , continue...")
        x_prev = x

    logging.info(f"Method ended on iteration {iter_no} with x0 value of {x[0]}, x1 value of {x[1]}")
    return x

### Вывод

In [5]:
eps = 1e-5

a = [0, 0]
b = [0.7, 0.7]

print("Iteration method:", iteration_method(a, b, eps))
print("Newton method:", newton_method(a, b, eps))

INFO:root:Inital x0 = 0.35, x1 = 0.35, q = 0.425531914893617
INFO:root:Iteration 1: x0 = 1.9691368667515217, x1 = 0.8544153281560676
INFO:root:1.2562139684224416 > 1e-05 , continue...
INFO:root:Iteration 2: x0 = 1.8083070665713716, x1 = 1.378548657144774
INFO:root:0.40611375161376345 > 1e-05 , continue...
INFO:root:Iteration 3: x0 = 1.4490008971302746, x1 = 1.3371847509079893
INFO:root:0.26791057220324754 > 1e-05 , continue...
INFO:root:Iteration 4: x0 = 1.487258196124378, x1 = 1.238084594067826
INFO:root:0.07868766551034106 > 1e-05 , continue...
INFO:root:Iteration 5: x0 = 1.5707152949952154, x1 = 1.2491158102991176
INFO:root:0.06235776719447639 > 1e-05 , continue...
INFO:root:Iteration 6: x0 = 1.5619570072382847, x1 = 1.2727659384653718
INFO:root:0.018681301827688675 > 1e-05 , continue...
INFO:root:Iteration 7: x0 = 1.5427465332589023, x1 = 1.270310114956977
INFO:root:0.014345786142164314 > 1e-05 , continue...
INFO:root:Iteration 8: x0 = 1.5447693069963526, x1 = 1.2649022831771384
IN

Iteration method: [1.547994861186502, 1.2663790861850406]
Newton method: [1.5479914435766509, 1.2663816528256782]
